In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import dash_daq as daq
import plotly.express as px
import pandas as pd
import xarray as xr
import datetime as dt
import numpy as np
import pytz
import cmocean as cmo
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

def cmocean_to_plotly(cmap, pl_entries=256):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

cbars = {"Temperature [°C]": cmocean_to_plotly(cmo.cm.balance),
        "Wind Speed [m/s]": cmocean_to_plotly(cmo.cm.speed),
        "Wind Direction [°]": cmocean_to_plotly(cmo.cm.phase),
        "Relative Humidity [%]": cmocean_to_plotly(cmo.cm.ice_r),
        "Pressure [hPa]": cmocean_to_plotly(cmo.cm.haline_r),
        "Local Time": cmocean_to_plotly(cmo.cm.solar),
        "Time [UTC]": cmocean_to_plotly(cmo.cm.solar)}

# App to check any boat day

boats = ["MSPolargirl", "MSBillefjord", "MSBerg"]
boat_data = {}

for boat in boats:
    print(boat)
    with xr.open_dataset(f"https://thredds.met.no/thredds/dodsC/met.no/observations/unis/mobile_AWS_{boat}_1min") as f:
        ds = f[["temperature","wind_speed_corrected","wind_direction_corrected",
                "relative_humidity","air_pressure", "longitude", "latitude"]]#, "exhaust_plume_influence"]]
        ds["temperature"] = ds["temperature"]#.where(ds["exhaust_plume_influence"] == 0)
        ds["relative_humidity"] = ds["relative_humidity"]#.where(ds["exhaust_plume_influence"] == 0)
        #ds = ds.drop_vars("exhaust_plume_influence")
        ds = ds.rename({"temperature": "Temperature [°C]",
                        "relative_humidity": "Relative Humidity [%]",
                        "air_pressure": "Pressure [hPa]",
                        "wind_speed_corrected": "Wind Speed [m/s]",
                        "wind_direction_corrected": "Wind Direction [°]",
                        "latitude": "Latitude [°N]",
                        "longitude": "Longitude [°E]"})
        boat_data[boat] = ds



app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

variables = list(boat_data[list(boat_data.keys())[0]].data_vars)
variables.remove('Longitude [°E]')
variables.remove('Latitude [°N]')

boat_dropdown = html.Div(dcc.Dropdown(list(boat_data.keys()), list(boat_data.keys()), id="boat-dropdown", multi=True))
variable_dropdown = html.Div(dcc.Dropdown(variables, "Temperature [°C]", id="variable-dropdown"))

time_select = html.Div(dcc.DatePickerSingle(id='time-picker',
                                            min_date_allowed=dt.date(2021, 8, 19),
                                            initial_visible_month=dt.date.today(),
                                            display_format='Y-M-D',
                                            month_format='Y-M-D',
                                            date=dt.date(2023,8,20)))      # dt.date.today()

utc_switch = html.Div(daq.BooleanSwitch(id='utc-switch', on=False, label="UTC", labelPosition="top"))

graph = dbc.Card(dcc.Graph(id="boat-tracks-plot"))

app.layout = dbc.Container([html.H2("Boat day", className="text-center"),
                            dbc.Row([dbc.Col(boat_dropdown, width=True),
                                     dbc.Col(variable_dropdown, width=True),
                                     dbc.Col(time_select, width="auto"),
                                     dbc.Col(utc_switch, width=1)]), 
                            dbc.Row(dbc.Col(graph))], 
                            fluid=True)

@app.callback(
    Output('boat-tracks-plot', 'figure'),
    Input('time-picker', 'date'),
    Input("boat-dropdown", "value"),
    Input("variable-dropdown", "value"),
    Input("utc-switch", "on")
)

def update_graph(plotdate, st, vari, switch_utc):
    if isinstance(st, str):
        st = [st]

    print("HI")

    if vari == None:
        df = []
        for s in st:
            try:
                df.append(boat_data[s][["Latitude [°N]", "Longitude [°E]"]].sel(time=plotdate).dropna("time").to_dataframe())
            except KeyError:
                pass
        try:
            df = pd.concat(df, axis=0)
        except ValueError:
            dr = pd.date_range(dt.datetime.strptime(plotdate, "%Y-%m-%d"), dt.datetime.strptime(plotdate, "%Y-%m-%d")+dt.timedelta(days=1), freq="1min", name="time")
            df = pd.DataFrame(np.ones((len(dr),2))*np.nan, index=dr, columns=pd.Index(["Latitude [°N]", "Longitude [°E]"]))
            df.loc[:,"Latitude [°N]"] = 78.228
            df.loc[:,"Longitude [°E]"] = 15.611
        df.index = df.index.tz_localize(tz='UTC')
        if not switch_utc:
            df.index = df.index.tz_convert("Europe/Oslo")
            vari = "Local Time"
        else:
            vari = "Time [UTC]"
        df[vari] = df.index.hour
        vari_time = vari
    
    else:
        if len(st) == 0:
            dr = pd.date_range(dt.datetime.strptime(plotdate, "%Y-%m-%d"), dt.datetime.strptime(plotdate, "%Y-%m-%d")+dt.timedelta(days=1), freq="1min", name="time")
            df = pd.DataFrame(np.ones((len(dr),3))*np.nan, index=dr, columns=pd.Index([vari, "Latitude [°N]", "Longitude [°E]"]))
            df.loc[:,"Latitude [°N]"] = 78.228
            df.loc[:,"Longitude [°E]"] = 15.611
        else:
            df = []
            for s in st:
                try:
                    df.append(boat_data[s][[vari, "Latitude [°N]", "Longitude [°E]"]].sel(time=plotdate).dropna("time").to_dataframe())
                except KeyError:
                    pass
            df = pd.concat(df, axis=0)
        df.index = df.index.tz_localize(tz='UTC')
        if not switch_utc:
            df.index = df.index.tz_convert("Europe/Oslo")
            df["Local Time"] = df.index
            vari_time = "Local Time"
        else:
            df["Time [UTC]"] = df.index
            vari_time = "Time [UTC]"

    
    fig = px.scatter_mapbox(df, lat="Latitude [°N]", lon="Longitude [°E]", color=vari, hover_data=vari_time, color_continuous_scale=cbars[vari], zoom=6)    
    print("HO")
    fig.update_layout(mapbox_style="stamen-terrain", margin={"r":0,"t":0,"l":0,"b":0}, autosize=True,
                  #mapbox_bounds={"west": 13., "east": 18., "south": 77.9, "north": 78.9}
                 )
    return fig


if __name__ == '__main__':
    app.run(debug=True, port=8051)

MSPolargirl
MSBillefjord
MSBerg
